# Auto loans default

The project is about analysis and predition based on the data regarding auto loans defaults. The goal is to find out what are the main factors impacting if the client deafults on the auto loan or not. It consist of exploratory data analysis, feauture engineering, some visualization and model creation and tuning. 

The data is taken from on the Kaggle's hackathons:
https://www.kaggle.com/datasets/saurabhbagchi/dish-network-hackathon?select=Data_Dictionary.csv


In [2]:
#Importing neccessery libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data

Since that was a contest, the data is already splitted into train and test, thus this part has been skipped. "data_dictionary is the description of columns". 


In [4]:
# Reading the data

data_dictionary = pd.read_csv('Data_Dictionary.csv')
train_data = pd.read_csv("Train_Dataset.csv")
test_data = pd.read_csv("Test_Dataset.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,7,8,16,17,18,19,20,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8,16,17,18,19,20,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


During the reading of the files the warning was triggered: 

"<i>DtypeWarning: Columns (1,7,8,16,17,18,19,20,35) have mixed types.Specify dtype option on import or set low_memory=False</i>". 

That already indicated that some data cleansing or feauture engineering would be required.

In [11]:
pd.options.display.max_colwidth = 150 # Option to see the entire content of the description
data_dictionary

,Variable,Description
0,ID,Client Loan application ID
1,Client_Income,Client Income in $
2,Car_Owned,Any Car owned by client before applying for the loan for another car (0 means No and 1 means otherwise)
3,Bike_Owned,Any bike owned by client (0 means No and 1 means otherwise)
4,Active_Loan,Any other active loan at the time of aplication of loan (0 means No and 1 means otherwise)
5,House_Own,Any house owned by client (0 means No and 1 means otherwise)
6,Child_Count,Number of children the client has
7,Credit_Amount,Credit amount of the loan in $
8,Loan_Annuity,Loan annuity in $
9,Accompany_Client,Who accompanied the client when client applied for the loan


There are 40 columns in total; Some of them are quite typical for credit risk analysis like income, loan amount, education, financial situation or number of issues. Additionally we also have information that potentially look irrelevant (e.g. day of the week and hour that client applied for a loan or age of ID document), but may surprisingly prove otherwise.

Last column ("Default") is to indicate the the client defaulted or not. This is going to be the target value for training and then predicting. 

### Data cleaning, feauture engineering



In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121856 entries, 0 to 121855
Data columns (total 40 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ID                          121856 non-null  int64  
 1   Client_Income               118249 non-null  object 
 2   Car_Owned                   118275 non-null  float64
 3   Bike_Owned                  118232 non-null  float64
 4   Active_Loan                 118221 non-null  float64
 5   House_Own                   118195 non-null  float64
 6   Child_Count                 118218 non-null  float64
 7   Credit_Amount               118224 non-null  object 
 8   Loan_Annuity                117044 non-null  object 
 9   Accompany_Client            120110 non-null  object 
 10  Client_Income_Type          118155 non-null  object 
 11  Client_Education            118211 non-null  object 
 12  Client_Marital_Status       118383 non-null  object 
 13  Client_Gender 

There is some work to be done to redefine types of some columns. Some columns will require creating dummy values.

In [22]:
# % of missing values for given columns, sorted

train_data.isnull().sum().sort_values(ascending=False)/len(train_data)*100 

Own_House_Age                 65.729221
Score_Source_1                56.488806
Social_Circle_Default         50.820641
Client_Occupation             34.003250
Score_Source_3                22.092470
Credit_Bureau                 15.214680
ID_Days                        4.897584
Score_Source_2                 4.666163
Population_Region_Relative     3.985852
Loan_Annuity                   3.948923
Client_Income_Type             3.037191
Client_Housing_Type            3.025702
Phone_Change                   3.006828
Application_Process_Hour       3.006007
House_Own                      3.004366
Loan_Contract_Type             2.996159
Employed_Days                  2.994518
Client_Education               2.991236
Child_Count                    2.985491
Active_Loan                    2.983029
Credit_Amount                  2.980567
Bike_Owned                     2.974002
Registration_Days              2.965796
Type_Organization              2.961692
Client_Income                  2.960051


to exclude ID, which does not carry informative value